In [ ]:
!pip install tensorflow_ranking
!pip install tf2onnx

In [3]:
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Reshape, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
import tensorflow as tf
import tensorflow_ranking as tfr
import tf2onnx

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
output_path = 'drive/MyDrive/CURIS/networks/'

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

11490434/11490434 [==============================] - 0s 0us/step


In [7]:
def eval_and_save(model):
  score = model.evaluate(x_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])
  model.summary()
  model.save(output_path + model.name + '.h5')
  model_proto, _ = tf2onnx.convert.from_keras(model, output_path=output_path + model.name + '.onnx')

10x2 fully connected

In [7]:
model = Sequential(name='mnist-10x2')
model.add(Flatten(name='input'))
model.add(Dense(10, name='dense_1', activation='relu'))
model.add(Dense(10, name='dense_2', activation='relu'))
model.add(Dense(10, name='logit'))
# model.add(Activation('softmax', name='output'))

In [ ]:
model.compile(# loss='categorical_crossentropy',
              # loss=tf.keras.losses.CategoricalCrossentropy(),
              loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

In [9]:
eval_and_save(model)

Test loss: 0.21554696559906006
Test accuracy: 0.9372000098228455
Model: "mnist-10x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 logit (Dense)               (None, 10)                110       
                                                                 
Total params: 8,070
Trainable params: 8,070
Non-trainable params: 0
_________________________________________________________________


30x2 fully connected

In [ ]:
model = Sequential(name='mnist-30x2')
model.add(Flatten(name='input'))
model.add(Dense(30, name='dense_1', activation='relu'))
model.add(Dense(30, name='dense_2', activation='relu'))
model.add(Dense(10, name='logit'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

In [13]:
eval_and_save(model)

Test loss: 0.12387928366661072
Test accuracy: 0.9656999707221985
Model: "mnist-30x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 30)                23550     
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 logit (Dense)               (None, 10)                310       
                                                                 
Total params: 24,790
Trainable params: 24,790
Non-trainable params: 0
_________________________________________________________________


100x2 fully connected

In [ ]:
model = Sequential(name='mnist-100x2')
model.add(Flatten(name='input'))
model.add(Dense(100, name='dense_1', activation='relu'))
model.add(Dense(100, name='dense_2', activation='relu'))
model.add(Dense(10, name='logit'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

In [15]:
eval_and_save(model)

Test loss: 0.09058046340942383
Test accuracy: 0.9796000123023987
Model: "mnist-100x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               78500     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 logit (Dense)               (None, 10)                1010      
                                                                 
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


simple cnn model

In [ ]:
model = Sequential(name='mnist-cnn-3x3')
model.add(Conv2D(4, (3, 3), name='conv_1', input_shape=(28, 28, 1)))
model.add(Conv2D(4, (3, 3), name='conv_2'))
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(10, name='logit'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

In [17]:
eval_and_save(model)

Test loss: 0.14827100932598114
Test accuracy: 0.960099995136261
Model: "mnist-cnn-3x3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 26, 26, 4)         40        
                                                                 
 conv_2 (Conv2D)             (None, 24, 24, 4)         148       
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 20)                46100     
                                                                 
 logit (Dense)               (None, 10)                210       
                                                                 
Total params: 46,498
Trainable params: 46,498
Non-trainable params: 0
___________________________________________________

cnn with max pooling and dropout

In [ ]:
model = Sequential(name='mnist-sota')
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dense(10, name='logit'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

In [9]:
eval_and_save(model)

Test loss: 0.028255069628357887
Test accuracy: 0.9939000010490417
Model: "mnist-sota"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0